# Imports

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
!pip install insightface onnxruntime-gpu --quiet

In [ ]:
!pip install facenet-pytorch ninja lpips torchmetrics --force-reinstall --no-cache-dir

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2-ada-pytorch")

In [ ]:
import os

# Create a folder for models
os.makedirs('models', exist_ok=True)

# Download the stylegan2-ada-pytorch FFHQ model (resolution 1024x1024)
# This is hosted by NVIDIA
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -O models/ffhq.pkl

print("Download complete.")

In [ ]:
import torch
import pickle
import copy
import dnnlib
import legacy # From the cloned repo

import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import lpips
import math
import random

import insightface
from insightface.app import FaceAnalysis

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Ensure deterministic behavior (might slow down slightly)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(0)

In [ ]:
# load dictionary of {filename: embedding_vector}
with open("embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

In [ ]:
class StyleGANGenerator(torch.nn.Module):
    def __init__(self, network_pkl):
        super(StyleGANGenerator, self).__init__()
        print(f'Loading network from "{network_pkl}"...')

        with dnnlib.util.open_url(network_pkl) as f:
            # Load the network from the pickle file
            network_dict = legacy.load_network_pkl(f)
            self.G = network_dict['G_ema'].to(device)
            self.D = network_dict['D'].to(device)
            self.D.eval()
            for param in self.D.parameters():
                param.requires_grad = False

        # Lock the weights (we never train the generator itself)
        self.G.eval()
        for param in self.G.parameters():
            param.requires_grad = False

        # Store useful constants
        self.w_dim = self.G.w_dim  # Usually 512
        self.num_ws = self.G.mapping.num_ws # Usually 18 for 1024x1024
        print(f'Loaded network! (w_dim: {self.w_dim}, num_ws: {self.num_ws})')

    def forward(self, w_plus_vector):
        """
        Input: w_plus_vector of shape (Batch, 18, 512)
        Output: Image tensor (Batch, 3, 1024, 1024) in range [-1, 1]
        """
        # synthesis() expects input to be split by layers, but w+ is already shaped correctly
        # noise_mode='const' means we don't add random noise to hair/pores every time (deterministic)
        img = self.G.synthesis(w_plus_vector, noise_mode='const')
        return img

    def get_mean_w(self, n_samples=4096, seed=0):
        """
        Get the average latent code (W space).
        Optimizing starting from the Mean Face is much faster/easier.
        """
        torch.manual_seed(seed)
        z = torch.randn(n_samples, self.G.z_dim, device=device)
        w = self.G.mapping(z, None) # Convert z to w
        w_avg = w.mean(0, keepdim=True)

        return w_avg

# Initialize the model
generator = StyleGANGenerator('models/ffhq.pkl')
print("Generator Loaded Successfully!")

In [ ]:
# 1. Get the mean latent code
w_mean = generator.get_mean_w()

# 2. Generate the image
with torch.no_grad():
    generated_img_tensor = generator(w_mean)

# 3. Convert from [-1, 1] range to [0, 1] for visualization
# StyleGAN output is (B, 3, H, W)
vis_img = (generated_img_tensor.clamp(-1, 1) + 1) / 2.0
vis_img = vis_img[0].cpu() # Take first item in batch

# 4. Show it
plt.imshow(vis_img.permute(1, 2, 0).numpy())
plt.axis('off')
plt.title("The Average Person (Mean Face)")
plt.show()

In [ ]:
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)
transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor()
])

In [ ]:
img_00000 = Image.open("00000.jpg").convert("RGB")
x_00000 = transform(img_00000).unsqueeze(0).to(device)
img_00001 = Image.open("00001.jpg").convert("RGB")
x_00001 = transform(img_00001).unsqueeze(0).to(device)
img_00002 = Image.open("00002.jpg").convert("RGB")
x_00002 = transform(img_00002).unsqueeze(0).to(device)

In [ ]:
emb_00000 = model(x_00000*2-1).detach().cpu()
emb_00001 = model(x_00001*2-1).detach().cpu()
emb_00002 = model(x_00002*2-1).detach().cpu()

In [ ]:
target_embedding = emb_00001.to(device)
target_image = x_00001

# General Functions

In [ ]:
class VGGPerceptualLoss(nn.Module):
    def __init__(self, resize=True):
        super(VGGPerceptualLoss, self).__init__()

        # Load VGG16
        vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).features

        # Slicing up to layer 16 (ReLU3_3) is standard.
        self.blocks = nn.Sequential(*list(vgg.children())[:16]).eval()

        # Freeze the model weights
        for param in self.blocks.parameters():
            param.requires_grad = False

        # VGG specific normalization
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1).to(device)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1).to(device)
        self.resize = resize

    def forward(self, generated_img, target_img):
        # Assuming the images are in [0, 1] range:
        gen_norm = (generated_img - self.mean) / self.std
        target_norm = (target_img - self.mean) / self.std

        # Extract features
        gen_features = self.blocks(gen_norm)
        target_features = self.blocks(target_norm)

        # Calculate L2 loss between the feature maps
        loss = torch.nn.functional.mse_loss(gen_features, target_features)
        return loss

perceptual_criterion = VGGPerceptualLoss().to(device)

In [ ]:
def display_grid_graphs(metrics_dict, n_cols=2, steps_log=None, log_scale_keys=None, figsize=None):
    """
    Plots multiple graphs in a grid.

    Args:
        metrics_dict (dict): Dictionary where Key is the Title and Value is the list of data.
        n_cols (int): Number of columns in the grid.
        steps_log (list): list of step jumps. If None, include all the steps.
        log_scale_keys (list): List of keys from metrics_dict that should be plotted in log scale.
        figsize (tuple): Optional custom size (width, height). If None, calculates automatically.
    """
    if steps_log is None:
        steps_log = list(range(len(next(iter(metrics_dict.values())))))

    if log_scale_keys is None:
        log_scale_keys = []

    # Calculate Grid Dimensions
    n = len(metrics_dict)
    n_rows = math.ceil(n / n_cols)

    # Auto-calculate figure size if not provided
    if figsize is None:
        figsize = (4 * n_cols, 3 * n_rows)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    if n == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    # Plot Data
    for i, (label, values) in enumerate(metrics_dict.items()):
        ax = axes[i]
        ax.plot(steps_log, values)

        ax.set_title(f"{label} per Step")
        ax.set_xlabel("Step")
        ax.set_ylabel(label)
        ax.grid(True, alpha=0.3)

        if label in log_scale_keys:
            ax.set_yscale('log')

    # Hide empty subplots (if n is not a perfect multiple of n_cols)
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def save_and_display_image(image, filename):
    image = image[0].permute(1, 2, 0).cpu().numpy()

    image_pil = transforms.ToPILImage()(image)
    image_pil.save(filename)

    plt.imshow(image)
    plt.axis('off')
    plt.show()

    # display(final_image)

In [ ]:
lpips_metric = lpips.LPIPS(net='vgg').to(device)
# < 0.25 high similarity
# > 0.7 different images

psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
# > 30 dB: High quality (hard to distinguish difference).
# 20-30 dB: Acceptable quality.
# < 20 dB: Poor quality (very noisy).

ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
# 1.0: Identical images.
# > 0.9: Very structurally similar.

In [ ]:
def evaluate_and_log(i, iterations, current_img, target_img, history, freq=20, grayscale=False):
    """
    Evaluates metrics and updates history lists in-place.

    Args:
        i (int): Current iteration.
        iterations (int): Total iterations.
        current_img (Tensor): The normalized image (output of tanh, [-1, 1]).
        target_img (Tensor): The target image ([0, 1]).
        history (tuple): (lpips_list, psnr_list, ssim_list, steps).
        freq (int): Log frequency.
    """
    if i % freq != 0 and i != iterations - 1:
        return

    lpips_list, psnr_list, ssim_list, steps = history

    with torch.no_grad():
        # Convert [-1, 1] -> [0, 1]
        val_img = (current_img * 0.5) + 0.5
        tgt_img = target_img

        # Clamp to ensure numerical stability (fix float errors like -0.0001 or 1.0001)
        val_img = val_img.clamp(0, 1)
        tgt_img = tgt_img.clamp(0, 1)

        if grayscale:
            val_img = transforms.functional.rgb_to_grayscale(val_img, num_output_channels=3)
            tgt_img = transforms.functional.rgb_to_grayscale(tgt_img, num_output_channels=3)

        lpips_list.append(lpips_metric(val_img, tgt_img).item())
        psnr_list.append(psnr_metric(val_img, tgt_img).item())
        ssim_list.append(ssim_metric(val_img, tgt_img).item())
        steps.append(i)

# exp 11

### Setup

In [ ]:
experiment_name = "exp11_gmi_b"

iterations = 1000

w_avg = generator.get_mean_w()
latent_code = w_avg.clone().detach().to(device)
latent_code.requires_grad = True

optimizer = optim.Adam([latent_code], lr=0.05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations)

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.002  # reg loss - deviation from average face

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, steps)

### Attack loop

In [ ]:
for i in tqdm(range(iterations)):
    optimizer.zero_grad()

    # Generate the image
    generated_image_1024 = generator(latent_code)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    # Penalize if the code gets too far from the average face (prevents "weird" artifacts)
    loss_reg = torch.mean((latent_code - w_avg) ** 2)
    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i, iterations, generated_image_160, target_image, history_lists, freq=20)
    # perceptual_input = (generated_image_160 * 0.5) + 0.5
    # perceptual_with_real = perceptual_criterion(perceptual_input, target_image)

    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)
    # perceptual_list.append(perceptual_with_real.item())

final_image = generated_image_160.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)
print("Inversion Complete.")

In [ ]:
display_grid_graphs({
    "Loss": loss_list,
    "Cosine Similarity": cosine_similarity_list
}, n_cols=3)

In [ ]:
display_grid_graphs({
    "LPIPS": lpips_list,
    "PSNR": psnr_list,
    "SSIM": ssim_list
}, n_cols=3, steps_log=steps)
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.
""")

In [ ]:
with torch.no_grad():
    final_high_res = generator(latent_code)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00000

In [ ]:
_lpips_list, _psnr_list, _ssim_list, _steps = [], [], [], []
_history_lists = (_lpips_list, _psnr_list, _ssim_list, _steps)

In [ ]:
evaluate_and_log(0, 1000, generated_image_160, target_image, _history_lists, freq=20, grayscale=False)
evaluate_and_log(0, 1000, generated_image_160, target_image, _history_lists, freq=20, grayscale=True)

In [ ]:
_history_lists

# exp 12 - decaying reg weight

### Setup

In [ ]:
experiment_name = "exp12_gmi_dacaying_reg_weight"

iterations = 1000

w_avg = generator.get_mean_w()
latent_code = w_avg.clone().detach().to(device)
latent_code.requires_grad = True

optimizer = optim.Adam([latent_code], lr=0.05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations)

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.005  # reg loss - deviation from average face
reg_weight_start = 0.05
reg_weight_end = 0.0

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, steps)

### Attack loop

In [ ]:
for i in tqdm(range(iterations)):
    optimizer.zero_grad()

    # Generate the image
    generated_image_1024 = generator(latent_code)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((latent_code - w_avg) ** 2)

    current_weight = reg_weight_start + (reg_weight_end - reg_weight_start) * (i / iterations)

    # Total Loss
    total_loss = loss_mse + (current_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i, iterations, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

final_image = generated_image_160.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)
print("Inversion Complete.")

In [ ]:
display_grid_graphs({
    "Loss": loss_list,
    "Cosine Similarity": cosine_similarity_list
}, n_cols=3)

In [ ]:
display_grid_graphs({
    "LPIPS": lpips_list,
    "PSNR": psnr_list,
    "SSIM": ssim_list
}, n_cols=3, steps_log=steps)
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.
""")

In [ ]:
with torch.no_grad():
    final_high_res = generator(latent_code)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00000

# exp 13 - optimizing w instead of w+

### Setup

In [ ]:
experiment_name = "exp12_gmi_mahalanobis_dist"

iterations = 1000

w_avg = generator.get_mean_w()
w_single = w_avg[:, 0, :].clone().detach()
latent_code = w_single.to(device)
latent_code.requires_grad = True

optimizer = optim.Adam([latent_code], lr=0.05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations)

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.005  # reg loss - deviation from average face
reg_weight_start = 0.05
reg_weight_end = 0.0

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, steps)

### Attack loop

In [ ]:
for i in tqdm(range(iterations)):
    optimizer.zero_grad()

    w_stack = latent_code.unsqueeze(1).repeat(1, 18, 1)

    # Generate the image
    generated_image_1024 = generator(w_stack)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((latent_code - w_single.to(device)) ** 2)

    current_weight = reg_weight_start + (reg_weight_end - reg_weight_start) * (i / iterations)

    # Total Loss
    total_loss = loss_mse + (current_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i, iterations, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

final_image = generated_image_160.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)
print("Inversion Complete.")

In [ ]:
display_grid_graphs({
    "Loss": loss_list,
    "Cosine Similarity": cosine_similarity_list
}, n_cols=3)

In [ ]:
display_grid_graphs({
    "LPIPS": lpips_list,
    "PSNR": psnr_list,
    "SSIM": ssim_list
}, n_cols=3, steps_log=steps)
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.
""")

In [ ]:
with torch.no_grad():
    final_high_res = generator(w_stack)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00001

# exp 14 - coarse to fine - w then w+

### Setup

In [ ]:
experiment_name = "exp14_gmi_coarse_to_fine"

iterations_1 = 25

w_avg = generator.get_mean_w(seed=15)
w_single = w_avg[:, 0, :].clone().detach()
latent_code = w_single.to(device)
latent_code.requires_grad = True

optimizer = optim.Adam([latent_code], lr=0.05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations_1)

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.002  # reg loss - deviation from average face
# reg_weight_start = 0.05
# reg_weight_end = 0.0

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, steps)

images = []

### Attack loop

In [ ]:
for i in tqdm(range(iterations_1)):
    optimizer.zero_grad()

    w_stack = latent_code.unsqueeze(1).repeat(1, 18, 1)

    # Generate the image
    generated_image_1024 = generator(w_stack)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((latent_code - w_single.to(device)) ** 2)

    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i, iterations_1, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations_1}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
        img = (generated_image_160.detach() * 0.5) + 0.5
        images.append(img[0].permute(1, 2, 0).cpu().numpy())

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

iterations_2 = 350
w_stack = latent_code.detach().unsqueeze(1).repeat(1, 18, 1)
w_plus = w_stack.clone()
w_plus.requires_grad = True
optimizer_w_plus = optim.Adam([w_plus], lr=0.025)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.25, total_iters=iterations_2)
reg_loss_weight = 0.002  # reg loss - deviation from average face

for i in tqdm(range(iterations_2)):
    optimizer_w_plus.zero_grad()

    # Generate the image
    generated_image_1024 = generator(w_plus)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((w_plus - w_avg) ** 2)

    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i + iterations_1, iterations_2 + iterations_1, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer_w_plus.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations_2}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
        img = (generated_image_160.detach() * 0.5) + 0.5
        images.append(img[0].permute(1, 2, 0).cpu().numpy())

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

final_image = generated_image_160.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)
print("Inversion Complete.")

In [ ]:
display_grid_graphs({
    "Loss": loss_list,
    "Cosine Similarity": cosine_similarity_list
}, n_cols=3)

In [ ]:
display_grid_graphs({
    "LPIPS": lpips_list,
    "PSNR": psnr_list,
    "SSIM": ssim_list
}, n_cols=3, steps_log=steps)
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.
""")

In [ ]:
with torch.no_grad():
    final_high_res = generator(w_stack)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:

with torch.no_grad():
    final_high_res = generator(w_plus)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00001

In [ ]:
for img in images:
    plt.imshow(img)
    plt.show()

# exp 15 - find best seed

### Setup

In [ ]:
experiment_name = "exp15_gmi_best_seed"

iterations_1 = 25

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.002  # reg loss - deviation from average face
# reg_weight_start = 0.05
# reg_weight_end = 0.0

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, steps)

images = []

### Attack loop

In [ ]:
w_stack_list = []
for seed in range(10, 25):
    print(f"Seed: {seed}")
    w_avg = generator.get_mean_w(seed=seed)
    w_single = w_avg[:, 0, :].clone().detach()
    latent_code = w_single.to(device)
    latent_code.requires_grad = True

    optimizer = optim.Adam([latent_code], lr=0.05)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations_1)

    for i in tqdm(range(iterations_1)):
        optimizer.zero_grad()

        w_stack = latent_code.unsqueeze(1).repeat(1, 18, 1)

        # Generate the image
        generated_image_1024 = generator(w_stack)

        # Resize for FaceNet (160x160)
        generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

        # Get Embedding
        current_embedding = model(generated_image_160)

        # Calculate loss
        loss_mse = mse_loss(current_embedding, target_embedding)
        loss_reg = torch.mean((latent_code - w_single.to(device)) ** 2)

        # Total Loss
        total_loss = loss_mse + (reg_loss_weight * loss_reg)

        cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
        evaluate_and_log(i, iterations_1, generated_image_160, target_image, history_lists, freq=20)

        total_loss.backward()
        optimizer.step()
        scheduler.step()

        if i == 0 or (i + 1) % 100 == 0:
            print(f"Step [{i+1}/{iterations_1}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
            img = (generated_image_160.detach() * 0.5) + 0.5
            images.append(img[0].permute(1, 2, 0).cpu().numpy())

        loss_list.append(total_loss.item())
        cosine_similarity_list.append(cos_sim)

    w_stack_list.append((seed, w_stack))

In [ ]:
with torch.no_grad():
    for seed, w_stack in w_stack_list:
        final_high_res = generator(w_stack)
        final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
        print(f"Seed: {seed}")
        save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00001

# exp 16 - adding discriminator loss

### Setup

In [ ]:
experiment_name = "exp16_gmi_discriminator_loss"

iterations_1 = 25

w_avg = generator.get_mean_w(seed=15)
w_single = w_avg[:, 0, :].clone().detach()
latent_code = w_single.to(device)
latent_code.requires_grad = True

optimizer = optim.Adam([latent_code], lr=0.05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations_1)

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.002  # reg loss - deviation from average face
# reg_weight_start = 0.05
# reg_weight_end = 0.0

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, steps)

images = []

### Attack loop

In [ ]:
for i in tqdm(range(iterations_1)):
    optimizer.zero_grad()

    w_stack = latent_code.unsqueeze(1).repeat(1, 18, 1)

    # Generate the image
    generated_image_1024 = generator(w_stack)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((latent_code - w_single.to(device)) ** 2)

    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i, iterations_1, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations_1}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
        img = (generated_image_160.detach() * 0.5) + 0.5
        images.append(img[0].permute(1, 2, 0).cpu().numpy())

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

iterations_2 = 350
w_stack = latent_code.detach().unsqueeze(1).repeat(1, 18, 1)
w_plus = w_stack.clone()
w_plus.requires_grad = True
optimizer_w_plus = optim.Adam([w_plus], lr=0.025)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.25, total_iters=iterations_2)
reg_loss_weight = 0.002  # reg loss - deviation from average face
d_loss_weight = 0.0001

for i in tqdm(range(iterations_2)):
    optimizer_w_plus.zero_grad()

    # Generate the image
    generated_image_1024 = generator(w_plus)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((w_plus - w_avg) ** 2)
    d_logits = generator.D(generated_image_1024, c=None)
    loss_discriminator = torch.nn.functional.softplus(-d_logits).mean()
    # print(f"loss_mse: {loss_mse}. loss_reg: {loss_reg}. loss_discriminator: {loss_discriminator}")

    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg) + (d_loss_weight * loss_discriminator)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log(i + iterations_1, iterations_2 + iterations_1, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer_w_plus.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations_2}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
        img = (generated_image_1024.detach().clamp(-1, 1) + 1) / 2.0
        images.append(img[0].permute(1, 2, 0).cpu().numpy())

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

final_image = generated_image_160.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)
print("Inversion Complete.")

In [ ]:
display_grid_graphs({
    "Loss": loss_list,
    "Cosine Similarity": cosine_similarity_list
}, n_cols=3)

In [ ]:
display_grid_graphs({
    "LPIPS": lpips_list,
    "PSNR": psnr_list,
    "SSIM": ssim_list
}, n_cols=3, steps_log=steps)
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.
""")

In [ ]:
with torch.no_grad():
    final_high_res = generator(w_stack)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:

with torch.no_grad():
    final_high_res = generator(w_plus)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00001

In [ ]:
for img in images:
    plt.imshow(img)
    plt.show()

# exp 17 - adding arcface metric

In [ ]:
class ArcFaceMetric:
    def __init__(self, device='cuda'):
        # Load the default 'buffalo_l' model pack (contains ResNet50 ArcFace)
        self.app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
        self.app.prepare(ctx_id=0, det_size=(640, 640))
        self.handler = self.app.models['recognition'] # The ArcFace recognition model
        self.device = device

    def get_embedding(self, tensor_img):
        """
        Input: Tensor [1, 3, H, W] in range [-1, 1] or [0, 1], RGB
        Output: Numpy Array [512]
        """
        # 1. Convert Tensor to Numpy Image [H, W, 3] in range [0, 255]
        # Assuming input is [-1, 1] (tanh output)
        if tensor_img.min() < 0:
            img = (tensor_img * 0.5 + 0.5)
        else:
            img = tensor_img

        img = img.clamp(0, 1).cpu().detach().squeeze(0).permute(1, 2, 0).numpy()
        img = (img * 255).astype(np.uint8)

        # 2. Convert RGB to BGR (InsightFace expects BGR via OpenCV)
        img_bgr = img[:, :, ::-1]

        # 3. Resize to 112x112 (ArcFace standard input)
        # Note: Usually we use alignment (warping), but for simple metric
        # on already cropped faces, resizing is acceptable.
        import cv2
        img_resized = cv2.resize(img_bgr, (112, 112))

        # 4. Get Embedding (blob is a formatting helper)
        blob = cv2.dnn.blobFromImage(img_resized, 1.0 / 127.5, (112, 112), (127.5, 127.5, 127.5), swapRB=True)
        # Note: InsightFace handler expects raw forward pass usually,
        # but calling .get_feat is safer if wrapping 'get'
        # We can simulate the forward pass directly on the handler:
        embedding = self.handler.get_feat(img_resized)

        return embedding.flatten()

    def compute_sim(self, tensor_gen, tensor_target):
        emb_gen = self.get_embedding(tensor_gen)
        emb_target = self.get_embedding(tensor_target)

        # Compute Cosine Similarity
        from numpy.linalg import norm
        sim = np.dot(emb_gen, emb_target) / (norm(emb_gen) * norm(emb_target))
        return sim

# Initialize ONCE (it takes time to load)
arcface_metric = ArcFaceMetric(device=device)

In [ ]:
def evaluate_and_log2(i, iterations, current_img, target_img, history, freq=20, grayscale=False):
    """
    Evaluates metrics and updates history lists in-place.

    Args:
        i (int): Current iteration.
        iterations (int): Total iterations.
        current_img (Tensor): The normalized image (output of tanh, [-1, 1]).
        target_img (Tensor): The target image ([0, 1]).
        history (tuple): (lpips_list, psnr_list, ssim_list, steps).
        freq (int): Log frequency.
    """
    if i % freq != 0 and i != iterations - 1:
        return

    lpips_list, psnr_list, ssim_list, id_score_list, steps = history

    with torch.no_grad():
        # Convert [-1, 1] -> [0, 1]
        val_img = (current_img * 0.5) + 0.5
        tgt_img = target_img

        # Clamp to ensure numerical stability (fix float errors like -0.0001 or 1.0001)
        val_img = val_img.clamp(0, 1)
        tgt_img = tgt_img.clamp(0, 1)

        if grayscale:
            val_img = transforms.functional.rgb_to_grayscale(val_img, num_output_channels=3)
            tgt_img = transforms.functional.rgb_to_grayscale(tgt_img, num_output_channels=3)

        lpips_list.append(lpips_metric(val_img, tgt_img).item())
        psnr_list.append(psnr_metric(val_img, tgt_img).item())
        ssim_list.append(ssim_metric(val_img, tgt_img).item())

        arcface_val_img = val_img * 2 - 1
        arcface_tgt_img = tgt_img * 2 - 1
        id_sim = arcface_metric.compute_sim(arcface_val_img, arcface_tgt_img)
        id_score_list.append(id_sim)

        steps.append(i)

### Setup

In [ ]:
experiment_name = "exp17_gmi_coarse_to_fine"

iterations_1 = 25

w_avg = generator.get_mean_w(seed=15)
w_single = w_avg[:, 0, :].clone().detach()
latent_code = w_single.to(device)
latent_code.requires_grad = True

optimizer = optim.Adam([latent_code], lr=0.05)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations_1)

mse_loss = torch.nn.MSELoss()
reg_loss_weight = 0.002  # reg loss - deviation from average face
# reg_weight_start = 0.05
# reg_weight_end = 0.0

loss_list = []
cosine_similarity_list = []
lpips_list = []
psnr_list = []
ssim_list = []
id_score_list = []
steps = []
history_lists = (lpips_list, psnr_list, ssim_list, id_score_list, steps)

images = []

### Attack loop

In [ ]:
for i in tqdm(range(iterations_1)):
    optimizer.zero_grad()

    w_stack = latent_code.unsqueeze(1).repeat(1, 18, 1)

    # Generate the image
    generated_image_1024 = generator(w_stack)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((latent_code - w_single.to(device)) ** 2)

    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log2(i, iterations_1, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations_1}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
        img = (generated_image_160.detach() * 0.5) + 0.5
        images.append(img[0].permute(1, 2, 0).cpu().numpy())

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

iterations_2 = 350
w_stack = latent_code.detach().unsqueeze(1).repeat(1, 18, 1)
w_plus = w_stack.clone()
w_plus.requires_grad = True
optimizer_w_plus = optim.Adam([w_plus], lr=0.025)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.25, total_iters=iterations_2)
reg_loss_weight = 0.002  # reg loss - deviation from average face

for i in tqdm(range(iterations_2)):
    optimizer_w_plus.zero_grad()

    # Generate the image
    generated_image_1024 = generator(w_plus)

    # Resize for FaceNet (160x160)
    generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    # Get Embedding
    current_embedding = model(generated_image_160)

    # Calculate loss
    loss_mse = mse_loss(current_embedding, target_embedding)
    loss_reg = torch.mean((w_plus - w_avg) ** 2)

    # Total Loss
    total_loss = loss_mse + (reg_loss_weight * loss_reg)

    cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    evaluate_and_log2(i + iterations_1, iterations_2 + iterations_1, generated_image_160, target_image, history_lists, freq=20)

    total_loss.backward()
    optimizer_w_plus.step()
    scheduler.step()

    if i == 0 or (i + 1) % 100 == 0:
        print(f"Step [{i+1}/{iterations_2}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
        img = (generated_image_160.detach() * 0.5) + 0.5
        images.append(img[0].permute(1, 2, 0).cpu().numpy())

    loss_list.append(total_loss.item())
    cosine_similarity_list.append(cos_sim)

final_image = generated_image_160.detach().cpu().squeeze(0)
final_embedding = current_embedding.detach().cpu().squeeze(0)
print("Inversion Complete.")

In [ ]:
display_grid_graphs({
    "Loss": loss_list,
    "Cosine Similarity": cosine_similarity_list
}, n_cols=3)

In [ ]:
display_grid_graphs({
    "LPIPS↓": lpips_list,
    "PSNR↑": psnr_list,
    "SSIM↑": ssim_list,
    "ArcFace↑": id_score_list
}, n_cols=2, steps_log=steps)
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.

    ArcFace:
        > 0.60: Excellent. It is definitely the same person.
        0.40 – 0.60: Good. It looks like a "sibling" or the same person in very different lighting.
        < 0.40: Failure. The optimizer fooled FaceNet, but ArcFace is not convinced.
""")

In [ ]:
with torch.no_grad():
    final_high_res = generator(w_stack)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
with torch.no_grad():
    final_high_res = generator(w_plus)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00001

# exp 18 - center of several seeds

In [ ]:
target_embedding = emb_00001.to(device)
target_image = x_00001

In [ ]:
img_00001

In [ ]:
# @title
def display_grid_graphs2(metrics_dict, n_cols=2, steps_log=None, log_scale_keys=None, figsize=None):
    """
    Plots multiple graphs in a grid. Supports multiple attempts per metric.

    Args:
        metrics_dict (dict):
            Key = Title.
            Value = List of lists (e.g., [[attempt1_data], [attempt2_data]])
                    OR single list (backward compatible).
        n_cols (int): Number of columns in the grid.
        steps_log (list): X-axis values. If None, auto-generated from data length.
        log_scale_keys (list): Keys to plot in log scale.
        figsize (tuple): Custom size.
    """

    # 1. Handle defaults
    if log_scale_keys is None:
        log_scale_keys = []

    # 2. Determine steps_log (X-axis) if not provided
    if steps_log is None:
        # Peek at the first item to determine length
        first_val = next(iter(metrics_dict.values()))
        if isinstance(first_val[0], list):
            # It's a list of lists, take length of first attempt
            data_len = len(first_val[0])
        else:
            # It's a flat list
            data_len = len(first_val)
        steps_log = list(range(data_len))

    # 3. Calculate Grid Dimensions
    n = len(metrics_dict)
    n_rows = math.ceil(n / n_cols)

    if figsize is None:
        figsize = (5 * n_cols, 4 * n_rows) # Slightly larger default for clarity

    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    if n == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    # 4. Plot Data
    for i, (label, data) in enumerate(metrics_dict.items()):
        ax = axes[i]

        # Check if data is "List of Lists" (Multiple Attempts) or "List" (Single Run)
        if isinstance(data[0], list):
            # --- Multiple Attempts Logic ---
            for attempt_idx, attempt_values in enumerate(data):
                # Matplotlib cycles colors automatically for each plot call
                ax.plot(steps_log, attempt_values, label=f"Attempt {attempt_idx + 1}")

            # Add legend to distinguish attempts
            # ax.legend(fontsize='small')

        else:
            # --- Single Run Logic (Old compatibility) ---
            ax.plot(steps_log, data)

        # Formatting
        ax.set_title(f"{label}") # Removed "per Step" to keep it clean
        ax.set_xlabel("Step")
        ax.set_ylabel(label)
        ax.grid(True, alpha=0.3)

        if label in log_scale_keys:
            ax.set_yscale('log')

    # 5. Hide empty subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

### Setup

In [ ]:
experiment_name = "exp18_several_seeds_center"

def run_attack(seed: int):
    print(f"Currently testing seed: {seed}")
    # Setup --------------------------------------------------------------------
    iterations_1 = 25

    w_avg = generator.get_mean_w(seed=seed)
    w_single = w_avg[:, 0, :].clone().detach()
    latent_code = w_single.to(device)
    latent_code.requires_grad = True

    optimizer = optim.Adam([latent_code], lr=0.05)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=iterations_1)

    mse_loss = torch.nn.MSELoss()
    reg_loss_weight = 0.002  # reg loss - deviation from average face
    # reg_weight_start = 0.05
    # reg_weight_end = 0.0

    loss_list = []
    cosine_similarity_list = []
    lpips_list = []
    psnr_list = []
    ssim_list = []
    id_score_list = []
    steps = []
    history_lists = (lpips_list, psnr_list, ssim_list, id_score_list, steps)

    images = []

    # Attack Loop --------------------------------------------------------------
    for i in tqdm(range(iterations_1)):
        optimizer.zero_grad()

        w_stack = latent_code.unsqueeze(1).repeat(1, 18, 1)

        # Generate the image
        generated_image_1024 = generator(w_stack)

        # Resize for FaceNet (160x160)
        generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

        # Get Embedding
        current_embedding = model(generated_image_160)

        # Calculate loss
        loss_mse = mse_loss(current_embedding, target_embedding)
        loss_reg = torch.mean((latent_code - w_single.to(device)) ** 2)

        # Total Loss
        total_loss = loss_mse + (reg_loss_weight * loss_reg)

        cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
        evaluate_and_log2(i, iterations_1, generated_image_160, target_image, history_lists, freq=1)

        total_loss.backward()
        optimizer.step()
        scheduler.step()

        if i == 0 or (i + 1) % 100 == 0:
            print(f"Step [{i+1}/{iterations_1}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
            img = (generated_image_160.detach() * 0.5) + 0.5
            images.append(img[0].permute(1, 2, 0).cpu().numpy())

        loss_list.append(total_loss.item())
        cosine_similarity_list.append(cos_sim)

    iterations_2 = 350
    w_stack = latent_code.detach().unsqueeze(1).repeat(1, 18, 1)
    # w_plus = w_stack.clone()
    # w_plus.requires_grad = True
    # optimizer_w_plus = optim.Adam([w_plus], lr=0.025)
    # scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.25, total_iters=iterations_2)
    # reg_loss_weight = 0.002  # reg loss - deviation from average face

    # for i in tqdm(range(iterations_2)):
    #     optimizer_w_plus.zero_grad()

    #     # Generate the image
    #     generated_image_1024 = generator(w_plus)

    #     # Resize for FaceNet (160x160)
    #     generated_image_160 = F.interpolate(generated_image_1024, size=(160, 160), mode='bilinear', align_corners=False)

    #     # Get Embedding
    #     current_embedding = model(generated_image_160)

    #     # Calculate loss
    #     loss_mse = mse_loss(current_embedding, target_embedding)
    #     loss_reg = torch.mean((w_plus - w_avg) ** 2)

    #     # Total Loss
    #     total_loss = loss_mse + (reg_loss_weight * loss_reg)

    #     cos_sim = nn.functional.cosine_similarity(current_embedding, target_embedding).item()
    #     evaluate_and_log2(i + iterations_1, iterations_2 + iterations_1, generated_image_160, target_image, history_lists, freq=20)

    #     total_loss.backward()
    #     optimizer_w_plus.step()
    #     scheduler.step()

    #     if i == 0 or (i + 1) % 100 == 0:
    #         print(f"Step [{i+1}/{iterations_2}], Loss: {total_loss.item():.6f}")#, Perc: {perceptual_with_real.item():.6f}")
    #         img = (generated_image_160.detach() * 0.5) + 0.5
    #         images.append(img[0].permute(1, 2, 0).cpu().numpy())

    #     loss_list.append(total_loss.item())
    #     cosine_similarity_list.append(cos_sim)

    final_image = generated_image_160.detach().cpu().squeeze(0)
    final_embedding = current_embedding.detach().cpu().squeeze(0)
    print("Inversion Complete.")

    return loss_list, cosine_similarity_list, lpips_list, psnr_list, ssim_list, id_score_list, steps, w_stack, None#, w_plus

In [ ]:
seed_info = []
for seed in range(100):
    loss_list, cosine_similarity_list, lpips_list, psnr_list, ssim_list, id_score_list, steps, w_stack, w_plus = run_attack(seed)
    seed_info.append((seed, loss_list, cosine_similarity_list, lpips_list, psnr_list, ssim_list, id_score_list, steps, w_stack, w_plus))

### Attack loop

In [ ]:
display_grid_graphs2({
    "Loss": [info[1] for info in seed_info],
    "Cosine Similarity": [info[2] for info in seed_info]
}, n_cols=3)

In [ ]:
display_grid_graphs2({
    "LPIPS↓": [info[3] for info in seed_info],
    "PSNR↑": [info[4] for info in seed_info],
    "SSIM↑": [info[5] for info in seed_info],
    "ArcFace↑": [info[6] for info in seed_info]
}, n_cols=2, steps_log=seed_info[0][7])
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.

    ArcFace:
        > 0.60: Excellent. It is definitely the same person.
        0.40 – 0.60: Good. It looks like a "sibling" or the same person in very different lighting.
        < 0.40: Failure. The optimizer fooled FaceNet, but ArcFace is not convinced.
""")

In [ ]:
with torch.no_grad():
    for info in seed_info:
        print(f"seed: {info[0]}")
        final_high_res = generator(info[8])
        final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
        save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
img_00001

In [ ]:
w_stack_all_attempts = torch.stack([info[8] for info in seed_info], dim=0)
w_stack_mean = torch.mean(w_stack_all_attempts, dim=0)
w_stack_mean

In [ ]:
with torch.no_grad():
    final_high_res = generator(w_stack_mean)
    final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
    save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
combined_data = torch.cat([w_stack_all_attempts, w_stack_mean.unsqueeze(0)], dim=0)
combined_data = combined_data[:, :, 0, :]
combined_data = combined_data.view(combined_data.shape[0], -1)
combined_data.size()

In [ ]:
n_samples = combined_data.shape[0]
perp = min(30, n_samples - 1) if n_samples > 1 else 1

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=perp, random_state=42, init='pca', learning_rate='auto')
projections = tsne.fit_transform(combined_data.cpu().numpy())

# 5. Split back apart
points_2d = projections[:-1] # All the regular items
mean_2d = projections[-1]    # The last item (The Average)

# 6. Plot
plt.figure(figsize=(10, 8))

# Plot all sample points
plt.scatter(points_2d[:, 0], points_2d[:, 1], c='blue', alpha=0.6, label='Samples')

# Plot the Average (Make it distinct)
plt.scatter(mean_2d[0], mean_2d[1], c='red', s=200, marker='X', label='High-Dim Average')

for i in range(n_samples - 1):
    plt.annotate(i, (points_2d[i, 0], points_2d[i, 1]), fontsize=14)

plt.title("Latent Space Distribution (t-SNE)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
_lpips_list = []
_psnr_list = []
_ssim_list = []
_id_score_list = []
_steps = []
_history_lists = (_lpips_list, _psnr_list, _ssim_list, _id_score_list, _steps)

generated_image_160 = F.interpolate(final_high_res, size=(160, 160), mode='bilinear', align_corners=False)

evaluate_and_log2(0, 1, generated_image_160, target_image, _history_lists, freq=1)

In [ ]:
display_grid_graphs2({
    "LPIPS↓": [info[3] for info in seed_info] + [[_history_lists[0]] * len(seed_info[0][7])],
    "PSNR↑": [info[4] for info in seed_info] + [[_history_lists[1]] * len(seed_info[0][7])],
    "SSIM↑": [info[5] for info in seed_info] + [[_history_lists[2]] * len(seed_info[0][7])],
    "ArcFace↑": [info[6] for info in seed_info] + [[_history_lists[3]] * len(seed_info[0][7])]
}, n_cols=2, steps_log=seed_info[0][7])
print("""
    lpips_metric:
        < 0.25 high similarity
        > 0.7 different images

    psnr_metric:
        > 30 dB: High quality (hard to distinguish difference).
        20-30 dB: Acceptable quality.
        < 20 dB: Poor quality (very noisy).

    ssim_metric:
        1.0: Identical images.
        > 0.9: Very structurally similar.

    ArcFace:
        > 0.60: Excellent. It is definitely the same person.
        0.40 – 0.60: Good. It looks like a "sibling" or the same person in very different lighting.
        < 0.40: Failure. The optimizer fooled FaceNet, but ArcFace is not convinced.
""")

In [ ]:
with torch.no_grad():
    for info in seed_info:
        if info[0] not in [62, 80, 1, 76, 26, 36, 10]:
            continue
        print(f"seed: {info[0]}")
        final_high_res = generator(info[8])
        final_img = (final_high_res.clamp(-1, 1) + 1) / 2.0
        save_and_display_image(final_img, f"{experiment_name}.png")

In [ ]:
# now we need to try to continue optimize the mean w_stack
# and find the closest one to him...